# CSV file with JSON columns

This notebook will take a csv file with one or more columns in json format and expand those json columns. I use this notebook to process some O365 Security and Compliance log files in the absence of the higher-tiered licensing that provide in-place analysis.

#### Requirements: The CSV file should have headers. Each cell within a JSON column should contain the same fields/names. Non-integer row indices are okay. Row indices are preserved.

In [160]:
import pandas as pd
import json

### The following cell identifies the CSV file to be processed

You can statically specify your file in the leading section or else enter it when prompted.

In [161]:
####################
#ENTER YOUR DATA HERE: Path to csv file
#If file path is not statically specified here, the user will be prompted on execution of cell.
csv_file_path = ""
####################

file_found = False
while file_found == False:

    #Prompt user for file path and name if not entered above statically
    if csv_file_path == "":
        print("Please specify the path and filename of your csv file:")
        csv_file_path = input()

    #Convert csv file into a dataframe
    try:
        csv_df = pd.read_csv(csv_file_path)
        file_found = True
        break
    except FileNotFoundError:
        print("File not found: Please check your csv file path and name and rerun this cell.")
    except pd.errors.EmptyDataError:
        print("Data is Empty: Please check the contents of your file and rerun this cell.")
    except pd.errors.ParserError:
        print("Parsing Error: Please check the format of your file contents and rerun this cell.")
    csv_file_path = ""
    
if csv_df.shape[0] < 1:
    print("Zero rows found. Please ensure your csv file has data in its rows.")

Please specify the path and filename of your csv file:
My Cats.csv


In [162]:
#Identify columns and preview dataframe
columns = csv_df.columns
print(columns)
csv_df

Index(['Name', 'What Goes In', 'What Comes Out'], dtype='object')


,Name,What Goes In,What Comes Out
0,Calina,"{""Food"":[""Freeze-dried Raw Turkey""],""Treats"":[...","{""Hungry"":""Chatter"", ""Lonely"":"""",""Happy"":""Loud..."
1,Tibbit,"{""Food"":[""Freeze-dried Raw Turkey""],""Treats"":[...","{""Hungry"":""Chatter"", ""Lonely"":""Meows"",""Happy"":..."
2,Lilly,"{""Food"":[""Freeze-dried Raw Turkey""],""Treats"":[...","{""Hungry"":""Loud Yowls"", ""Lonely"":""Loudest Yowl..."
3,Ewok,"{""Food"":[""Freeze-dried Raw Turkey""],""Treats"":[...","{""Hungry"":"""", ""Lonely"":"""",""Happy"":""Quiet Purrs""}"
4,Wookie,"{""Food"":[""Freeze-dried Raw Turkey""],""Treats"":[...","{""Hungry"":""Tiny Meows"", ""Lonely"":"""",""Happy"":""S..."
5,Hina,"{""Food"":[""Freeze-dried Raw Turkey""],""Treats"":[...","{""Hungry"":""Pitiful Mews"", ""Lonely"":""Louder Pit..."


### The following cell determines which columns contain JSON and should be expanded

You can statically specify the columns, allow the code to autodect (based on the contents of the first row), or indicate column by column which to classify as json columns.

If you leave json_columns as an empty array, you will be presented with the columns that were autodetected as json and you can press enter (submit an empty response to the input request) to accept that default and continue. If you submit a non-empty response, you will be prompted to identify each column. You may also consider stopping the cell after the columns have been autodetected so you can copy and paste the columns into the leading section if you have only a small tweak to make.

In [163]:
####################
#ENTER YOUR DATA HERE: Specify json columns as a list of column names (strings).
#Columns are printed from the previous cell for review. User will be prompted if not specified here.
json_columns = []
####################

if len(json_columns) == 0:
    for col in columns:
        if type(csv_df.iloc[0][col]) == str:
            try:
                json.loads(csv_df.iloc[0][col])
                json_columns.append(col)
            except json.JSONDecodeError: 
                continue
print("The following have been identified as json columns. Please rerun this cell if not correct.")
print(json_columns)
print("Press Enter to accept, otherwise enter any character to manually choose columns instead.")
manual_select = input()
if manual_select != "":
    json_columns = []
    #Prompt user to identify json columns if not identified statically above
    print("For each column, enter 1 to identify a json column and 0 otherwise.")
    for col in columns:
        print(col,end=": ")
        response = input()
        if response == '1':
            json_columns.append(col)
    print("The following have been identified as json columns. Please rerun this cell if not correct.")
    print(json_columns)

The following have been identified as json columns. Please rerun this cell if not correct.
['What Goes In', 'What Comes Out']
Press Enter to accept, otherwise enter any character to manually choose columns instead.



In [164]:
#Construct a new column list with extra columns to accomodate the json expansions
new_columns = []
for col in columns:
    if col in json_columns:
        json_string = csv_df.iloc[0][col]
        example = json.loads(json_string)
        sub_columns = list(example.keys())
        for sub_col in sub_columns:
            #Prefix columns derived from json expansion with original column name
            new_columns.append(col+"_"+sub_col)
    else:
        new_columns.append(col)

#Construct an empty dataframe with new column names
json_expanded_df = pd.DataFrame(columns = new_columns)
json_expanded_df

,Name,What Goes In_Food,What Goes In_Treats,What Goes In_Oral Fixations,What Comes Out_Hungry,What Comes Out_Lonely,What Comes Out_Happy


In [165]:
new_row = []
for index, row in csv_df.iterrows():
    for col, value in csv_df.loc[index].items():
        if col in json_columns:
            new_row.extend(list(json.loads(value).values())) 
        else:
            new_row.append(value)
    json_expanded_df.loc[index] = new_row
    new_row = []
    
json_expanded_df

,Name,What Goes In_Food,What Goes In_Treats,What Goes In_Oral Fixations,What Comes Out_Hungry,What Comes Out_Lonely,What Comes Out_Happy
0,Calina,[Freeze-dried Raw Turkey],"[Chicken, Duck, Turkey, Salmon]","[Bananas, Potatoes, Upholstery]",Chatter,,Loud Purrs
1,Tibbit,[Freeze-dried Raw Turkey],"[Chicken, Duck, Turkey, Salmon]",[],Chatter,Meows,Purrs
2,Lilly,[Freeze-dried Raw Turkey],"[Duck, Turkey]","[Plastic Bags, Plastic Wrap]",Loud Yowls,Loudest Yowls,Loud Purrs
3,Ewok,[Freeze-dried Raw Turkey],"[Chicken, Duck, Turkey]",[People who aren't petting enough],,,Quiet Purrs
4,Wookie,[Freeze-dried Raw Turkey],"[Chicken, Duck, Turkey, Salmon]",[],Tiny Meows,,Sub-sonic Purrs
5,Hina,[Freeze-dried Raw Turkey],"[Chicken, Duck, Turkey, Salmon]","[String, Cardboard, Paper, Everything]",Pitiful Mews,Louder Pitiful Mews,Asserively Dominant Purrs


In [166]:
# Append _json_expanded to original filename
if len(csv_file_path) > 4 and csv_file_path[len(csv_file_path)-4:] == ".csv":
    csv_json_expanded_path = csv_file_path[:len(csv_file_path)-4]+'_json_expanded.csv'
else:
    print('There was a problem modifying the file path. The csv file will be saved in the root directory for jupyter notebooks as csv_json_expanded.csv instead.')
    csv_json_expanded_path = 'csv_json_expanded.csv'

json_expanded_df.to_csv(csv_json_expanded_path)